# TensorBoard

**Tensorboard** is the interface used to visualize the graph and other tools to understand, debug, and optimize the model. 

The panel contains different tabs, which are linked to the level of information you add when you run the model. 

- Scalars: Show different useful information during the model training
- Graphs: Show the model
- Histogram: Display weights with a histogram
- Distribution: Display the distribution of the weight
- Projector: Show Principal component analysis and T-SNE algorithm. The technique uses for dimensionality reduction


## Graphs 

By looking at the graph, you can understand how the model work.

- Enqueue the data to the model: Push an amount of data equal to the batch size to the model, i.e., Number of data feed after each iteration
- Feed the data to the Tensors
- Train the model
- Display the number of batches during the training. Save the model on the disk.
    
The basic idea behind tensorboard is that neural network can be something known as a black box and we need a tool to inspect what's inside this box. You can imagine tensorboard as a flashlight to start dive into the neural network.

It helps to understand the dependencies between operations, how the weights are computed, displays the loss function and much other useful information. When you bring all these pieces of information together, you have a great tool to debug and find how to improve the model. 

In [29]:
import tensorflow as tf
from math import sqrt

# For reference use the TensorFlow documentation at
# - https://www.tensorflow.org/api_docs/python/tf

# Function to comute 
# ( bias * param1 ) * sqrt( weight / param2)

params = {'param1':0.2, 'param2':4.0, 'bias':2.0, 'weight':8.0}
print(params)

# Input Parameters (scalars)
bias = tf.placeholder(tf.float32, name='bias')
weight = tf.placeholder(tf.float32, name='weight')

# Constant Values (hyper-parameters)
param1 = tf.constant(params['param1'], name='param1')
param2 = tf.constant(params['param2'], name='param2')

# Computation graph
#   div and multiply operation allows to set scalars, vector or matrices using tf optimizations
left = tf.multiply(bias, param1)
right = tf.sqrt(tf.div(weight, param2))
function =  tf.multiply(left, right)

# Create the tensor flow session
with tf.Session() as session:
    # Create the FileWriter for the graph
    writer = tf.summary.FileWriter(".outputs/output", session.graph)

    # Build the symbolic model (pipeline), execute the graph, and print the result
    tfOutput = session.run(function, {bias:params['bias'], weight:params['weight']})
    print('From TensorFlow = {}'.format(tfOutput))

    expectedOutput = ( params['bias'] * params['param1'] ) * sqrt( params['weight'] / params['param2'])
    print('From Expected = {}'.format(expectedOutput))
    # Close writes at the end
    writer.close()


{'param1': 0.2, 'param2': 4.0, 'bias': 2.0, 'weight': 8.0}
From TensorFlow = 0.5656854510307312
From Expected = 0.5656854249492381


Use the following command from this notebook to open the tensorboard, using previous log folder path

    tensorboard --logdir=.outputs/output
    
Use the browser with URL provided

## Advanced

Sometimes the graph can be very large and quite complex. In this case is useful to create `scopes` and `named` operations in orde to make the graph understandable.

In [31]:
import tensorflow as tf
from math import sqrt

# For reference use the TensorFlow documentation at
# - https://www.tensorflow.org/api_docs/python/tf

# Function to comute 
# ( bias * param1 ) * sqrt( weight / param2)

params = {'param1':0.2, 'param2':4.0, 'bias':2.0, 'weight':8.0}
print(params)

# Input Parameters (scalars)
bias = tf.placeholder(tf.float32, name='bias')
weight = tf.placeholder(tf.float32, name='weight')

# Constant Values (hyper-parameters)
param1 = tf.constant(params['param1'], name='param1')
param2 = tf.constant(params['param2'], name='param2')

# Computation graph
#   div and multiply operation allows to set scalars, vector or matrices using tf optimizations
with tf.name_scope("function"):
    with tf.name_scope("left_side"):
        left = tf.multiply(bias, param1, name='multiply') 
    with tf.name_scope("right_side"):
        right = tf.sqrt(tf.div(weight, param2, name='divide'),name='sqrt')
    function =  tf.multiply(left, right, name='multiply')

# Create the tensor flow session
with tf.Session() as session:
    # Create the FileWriter for the graph
    writer = tf.summary.FileWriter(".outputs/output2", session.graph)

    # Build the symbolic model (pipeline), execute the graph, and print the result
    tfOutput = session.run(function, {bias:params['bias'], weight:params['weight']})
    print('From TensorFlow = {}'.format(tfOutput))

    expectedOutput = ( params['bias'] * params['param1'] ) * sqrt( params['weight'] / params['param2'])
    print('From Expected = {}'.format(expectedOutput))
    # Close writes at the end
    writer.close()

#Use tensorboard --logdir=.outputs/output2

{'param1': 0.2, 'param2': 4.0, 'bias': 2.0, 'weight': 8.0}
From TensorFlow = 0.5656854510307312
From Expected = 0.5656854249492381


Use the following command from this notebook to open the tensorboard, using previous log folder path

    tensorboard --logdir=.outputs/output2

Use the browser with URL provided


## Summaries

Appart from scalars and graphs, it can be visualized summaries for the trained model.

In [32]:
import tensorflow as tf
import numpy as np

def variable_summaries(var):
    with tf.name_scope('summaries'):
        mean = tf.reduce_mean(var)
        tf.summary.scalar('mean', mean)
        with tf.name_scope('stddev'):
            stddev = tf.sqrt(tf.reduce_mean(tf.square(var - mean)))
            tf.summary.scalar('stddev', stddev)
            tf.summary.scalar('max', tf.reduce_max(var))
            tf.summary.scalar('min', tf.reduce_min(var))
            tf.summary.histogram('histogram', var)

# x and y are placeholders for our training data
x = tf.placeholder("float")
y = tf.placeholder("float")
# w is the variable storing our values. It is initialized with starting "guesses"
# w[0] is the "a" in our equation, w[1] is the "b"
w = tf.Variable([1.0, 2.0], name="w")
variable_summaries(w)
# Our model of y = a*x + b
with tf.name_scope('model'):
    y_model = tf.multiply(x, w[0]) + w[1]
    tf.summary.histogram('pre_activations', y_model)

# Our error is defined as the square of the differences
with tf.name_scope('error'):
    error = tf.square(y - y_model)
tf.summary.scalar('error', error)

# The Gradient Descent Optimizer does the heavy lifting
train_op = tf.train.GradientDescentOptimizer(0.01).minimize(error)

# Normal TensorFlow - initialize values, create a session and run the model
model = tf.global_variables_initializer()

with tf.Session() as session:
    merged_summary = tf.summary.merge_all()

     # Create the FileWriter for the graph
    writer = tf.summary.FileWriter(".outputs/output3", session.graph)

    session.run(model)
    for i in range(1000):
        x_value = np.random.rand()
        y_value = x_value * 2 + 6
        _, summary  = session.run([train_op, merged_summary], feed_dict={x: x_value, y: y_value})
         # Add output for merged variables into the summary (tensorboard)
        writer.add_summary(summary, i)

    w_value = session.run(w)
    print("Predicted model: {a:.3f}x + {b:.3f}".format(a=w_value[0], b=w_value[1]))

    # Close writes at the end
    writer.close()

Predicted model: 2.272x + 5.859
